https://www.dataquest.io/blog/settingwithcopywarning/  
の内容を検証するためのNotebook。


http://www.modelingonlineauctions.com/datasets  
の中にある「Xbox 3-day auctions.csv」をダウンロードしたい。  
しかし、使用しているOSがWindowsなので、!wget も !curlも使えない。  
よってpythonの機能でダウンロードしよう。  
……と思ったら、pandasのread_csvにURLを渡してやればできるんだな。簡単な話だった。

In [1]:
import pandas as pd

In [2]:
csv_file_path = 'xbox-3-day-auctions.csv'

if False: # todo: あとで条件を「csvファイルが存在しないときは」にしておこう
    url = 'http://www.modelingonlineauctions.com/datasets/Xbox%203-day%20auctions.csv?attredirects=0&d=1'
    data = pd.read_csv(url)
    data.to_csv(csv_file_path)
else:
    data = pd.read_csv(csv_file_path)

In [3]:
data.head()

,Unnamed: 0,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
0,0,8213034705,95.0,2.927373,jake7870,0,95.0,117.5
1,1,8213034705,115.0,2.943484,davidbresler2,1,95.0,117.5
2,2,8213034705,100.0,2.951285,gladimacowgirl,58,95.0,117.5
3,3,8213034705,117.5,2.998947,daysrus,10,95.0,117.5
4,4,8213060420,2.0,0.065266,donnie4814,5,1.0,120.0


# Chained Assignment

In [4]:
data[data['bidder'] == 'parakeet2004']

,Unnamed: 0,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
6,6,8213060420,3.00,0.186539,parakeet2004,5,1.0,120.0
7,7,8213060420,10.00,0.186690,parakeet2004,5,1.0,120.0
8,8,8213060420,24.99,0.187049,parakeet2004,5,1.0,120.0


In [5]:
data[data['bidder'] == 'parakeet2004']['bidderrate'] = 100

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


**SettingWithCopyWarning** が発生した。  
今回の場合、元の表の値は変わっていない。

In [6]:
data[data['bidder'] == 'parakeet2004']

,Unnamed: 0,auctionid,bid,bidtime,bidder,bidderrate,openbid,price
6,6,8213060420,3.00,0.186539,parakeet2004,5,1.0,120.0
7,7,8213060420,10.00,0.186690,parakeet2004,5,1.0,120.0
8,8,8213060420,24.99,0.187049,parakeet2004,5,1.0,120.0
